In [2]:
import pandas as pd
import numpy as np
import datetime
import pickle
import os

with open('hw_2_pickle_data_col.pkl', 'rb') as f:
    data_col = pickle.load(f)
    
GENDER_col_list, DAY_col_list, TIME_col_list, ACT_col_list = data_col

column_list = ['CUS_ID'] + GENDER_col_list + DAY_col_list + TIME_col_list + ACT_col_list

# 2-1. TIME_ID 를 사용하여 각 인터넷 사용 기록의 시간과 요일에 대한 열을 만든다.
## 전처리를 위한 전처리 함수 get_days 함수를 만들었다. datetime에서 인덱스를 반환할 때 각 요일의 문자열로 반환할 수 있도록.
## 데이터프레임의 TIME은 object, 즉 str 자료형인데, 차례대로 앞 네 자리, 두 자리, 두자리, 두자리 씩 연, 월, 일, 시각에 대한 정보 보유.
## 따라서 슬라이싱을 통해 분리하고, 앞서 말한 get_days 함수에 넣어 요일 데이터프레임, 시간 데이터프레임을 위한 리스트 만듦.

In [6]:
# 2. 과제
# 2-1. TIME_ID 를 사용하여 각 인터넷 사용 기록의 시간과 요일에 대한 열을 만든다.
#   (datetime 라이브러리를 사용하면 요일을 알아낼 수 있음. 다른 라이브러리를 사용해도 됨.)
#    ex)
#     CUS_ID    TIME_ID       ACT_NM     TIME_HOUR     DAY
#       9999   2012070412      일간지       12h         WED
#       9999   2012072414       검색        14h         TUE
#       9999   2012081613       검색        13h         THU
#       9999   2012092512       검색        12h         TUE
#       9999   2012101215       포털        15h         FRI

raw_data = pd.read_csv('hw_2_raw_data.csv', encoding="cp949")
label_data = pd.read_csv('hw_2_label_data.csv', encoding= "cp949")

meta_data = pd.merge(raw_data, label_data)

TIME_HOUR = []
DAY = []
meta_data['TIME'] = list(map(str,meta_data['TIME_ID']))

def get_days(yyyy, mm, dd):
    days = ['MON','TUE','WED','THU', 'FRI', 'SAT', 'SUN']
    return days[datetime.date(yyyy,mm,dd).weekday()]

for timeline in meta_data['TIME']:
    y = int(timeline[:4])
    m = int(timeline[4:6])
    d = int(timeline[6:8])
    h = str(int(timeline[8:]))+'h'
    TIME_HOUR.append(h)
    DAY.append(get_days(y,m,d))

meta_data['TIME_HOUR'] = TIME_HOUR
meta_data['DAY'] = DAY
#meta_data_sub = meta_data[['CUS_ID', 'GENDER']]
meta_data.drop(columns={'GENDER', 'AGE', 'AGE_CLASS', 'TIME'})

,CUS_ID,TIME_ID,ACT_NM,TIME_HOUR,DAY
0,1176,2012083016,학술정보,16h,THU
1,1176,2013030418,가격비교,18h,MON
2,1176,2012083119,포털,19h,FRI
3,1176,2012100510,포털,10h,FRI
4,1176,2012120617,교통,17h,THU
...,...,...,...,...,...
219971,34,2013012917,식품/건강 쇼핑몰,17h,TUE
219972,34,2013013010,블로그/SNS,10h,WED
219973,34,2013013018,취미/스포츠 쇼핑몰,18h,WED
219974,34,2013020709,포털,9h,THU


In [6]:
# 2-2. 개인별로 요일당, 시간당, 사이트 종류당 접속 횟수를 카운트한다.
#    ex)
#     9999 ->
#      요일 :  WED    1
#             TUE    2
#             THU    1
#             FRI    1
#     
#      시간 :  12h    2
#             14h    1
#             13h    1
#             15h    1
#     
#      종류 :  일간지    1
#             검색    3
#             포털    1

# 2-3. 위의 데이터에 개인별 총 접속횟수로 나누고, 성별 데이터를 포함하여 DataFrame 형식으로 save_df를 생성한다.
#    ex)
#     9999 ->
#         GENDER                   F
#         MON                      0
#         TUE                      0.4
#        12h                       0.4
#        13h                       0.2
#        14h                       0.2
#        15h                       0.2
#        16h                       0
#        일간지                    0.2
#        검색                      0.6
#        포털                      0.2
#        학술정보                  0
#        취업                      0
#        ......(생략)     
# =============================================================================

days = ['MON','TUE','WED','THU', 'FRI', 'SAT', 'SUN']

data_for_submission_1 = meta_data.groupby(['CUS_ID','DAY']).size() / meta_data.groupby(['CUS_ID']).size()
data_for_submission_1 = data_for_submission_1.unstack().replace(np.nan, 0)
data_for_submission_1 = data_for_submission_1[days]

data_for_submission_2 = meta_data.groupby(['CUS_ID','TIME_HOUR']).size() / meta_data.groupby(['CUS_ID']).size()
data_for_submission_2 = data_for_submission_2.unstack().replace(np.nan, 0)
#data_for_submission_2.columns = TIME_col_list

data_for_submission_3 = meta_data.groupby(['CUS_ID','ACT_NM']).size() / meta_data.groupby(['CUS_ID']).size()
data_for_submission_3 = data_for_submission_3.unstack().replace(np.nan, 0)
#data_for_submission_3.columns = ACT_col_list
#data_for_submission_3 = data_for_submission_3.rename_axis('CUS_ID').reset_index()

print(data_for_submission_1.head(3))
print(data_for_submission_2.head(3))
print(data_for_submission_3.head(3))

DAY          MON       TUE       WED       THU       FRI       SAT  SUN
CUS_ID                                                                 
2       0.176644  0.246489  0.160569  0.234109  0.170916  0.011271  0.0
3       0.220423  0.186444  0.174880  0.214019  0.160292  0.043942  0.0
8       0.210991  0.190638  0.225237  0.223202  0.149932  0.000000  0.0
TIME_HOUR  00h       01h       02h      03h       04h       05h       06h  \
CUS_ID                                                                      
2          0.0  0.001109  0.000554  0.00037  0.000185  0.002772  0.000739   
3          0.0  0.000000  0.000000  0.00000  0.000000  0.000000  0.000000   
8          0.0  0.000000  0.000000  0.00000  0.000000  0.000000  0.000000   

TIME_HOUR       07h       08h       09h  ...       14h       15h       16h  \
CUS_ID                                   ...                                 
2          0.001293  0.004065  0.039542  ...  0.094235  0.088692  0.077790   
3          0.008895 

# 각 데이터프레임을 분리해서 만들었고, 이를 
## 처음에는 count() 메서드를 통해 데이터프레임을 만들었는데, 이 메서드의 경우 기본 열 정보를 꼭 넣어야 해서 이중 컬럼이 형성돼 복잡해짐.
## size() 메서드에 대한 피드백을 얻어 수정했고, 훨씬 간단하게 데이터프레임을 만들 수 있었음(size 메서드는 인덱스만 설정하면 되기에 powerful).
## unstack을 통해 이중인덱스에서 소분류로 된 인덱스를 열로 치환할 수 있었다. 결측값은 replace 메서드를 통해 0으로 치환.
## meta_data.groupby(['CUS_ID']).size()로 각각 나누지 않았다면 2-2의 답안, 나누면 2-3의 답안.


## 제출할 데이터프레임에는 GENDER 열이 추가되어 있었다. 이를 위해 CUS_ID와 GENDER를 열로 갖는 데이터프레임 생성. 중복값을 메서드를 통해 제거.
## 한 번에 concat을 하려는데 왜인지는 모르겠지만 CUS_ID가 한 번에 키 값으로 인식되지 않았다. 그래서 중간 과정을 거치고, merge를 통해 키값을 CUS_ID 열로 해 병합했다.

In [8]:
data_for_submission_4 = meta_data[['CUS_ID', 'GENDER']]
data_for_submission_4 = data_for_submission_4.sort_values(['CUS_ID']).drop_duplicates(ignore_index = True)

data_for_submission_final = pd.concat([data_for_submission_1, data_for_submission_2, data_for_submission_3], axis=1, join="outer")
data_for_submission_final.reset_index()
data_for_submission_final = pd.merge(data_for_submission_4, data_for_submission_final, how='outer', on='CUS_ID')
data_for_submission_final

,CUS_ID,GENDER,MON,TUE,WED,THU,FRI,SAT,SUN,00h,...,학교,학교커뮤니티,학생/교과교육,학술정보,학원,해외지역정보,화장품/미용,화장품/미용 쇼핑몰,화학,환경
0,2,F,0.176644,0.246489,0.160569,0.234109,0.170916,0.011271,0.000000,0.000000,...,0.000185,0.000000,0.000000,0.000924,0.000185,0.0,0.000000,0.001663,0.0,0.000000
1,3,F,0.220423,0.186444,0.174880,0.214019,0.160292,0.043942,0.000000,0.000000,...,0.000178,0.000000,0.000000,0.001779,0.000890,0.0,0.002313,0.000712,0.0,0.000000
2,8,M,0.210991,0.190638,0.225237,0.223202,0.149932,0.000000,0.000000,0.000000,...,0.000678,0.000000,0.000000,0.000000,0.000000,0.0,0.002035,0.074627,0.0,0.000000
3,23,F,0.183091,0.207220,0.198011,0.223061,0.144962,0.017683,0.025972,0.006815,...,0.000184,0.000000,0.147357,0.004237,0.003684,0.0,0.000184,0.000184,0.0,0.000000
4,32,F,0.220423,0.186444,0.174880,0.214019,0.160292,0.043942,0.000000,0.000000,...,0.000178,0.000000,0.000000,0.001779,0.000890,0.0,0.002313,0.000712,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,2261,F,0.170525,0.145680,0.189159,0.138340,0.146810,0.102767,0.106719,0.000000,...,0.000000,0.000000,0.000000,0.002259,0.000000,0.0,0.000000,0.000000,0.0,0.000000
66,2323,M,0.197151,0.196547,0.194374,0.202584,0.198117,0.007364,0.003863,0.000000,...,0.003139,0.000000,0.000000,0.007364,0.000000,0.0,0.000000,0.000000,0.0,0.000121
67,2367,M,0.220732,0.181336,0.176865,0.170439,0.184968,0.049455,0.016206,0.001956,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000279,0.000279,0.0,0.000000
68,2382,M,0.138524,0.155020,0.129656,0.108372,0.135509,0.159808,0.173111,0.085491,...,0.000000,0.000000,0.000000,0.009933,0.000177,0.0,0.000532,0.000000,0.0,0.000000


# 앞서 푼 풀이들을 이어붙이기만 하면 된다.

In [63]:
# 2-4. 생성한 save_df를 'hw_2_data_본인이름.csv'로 저장한다.
#       'hw_2_ex_data.csv'처럼 데이터 저장
# 
# 
# 
# 아래 코드 함수의 save_df를 작성하는 부분을 완성하시기 바랍니다.

import pandas as pd
import numpy as np
import datetime
import pickle
import os


data_file_path = '폴더 위치 입력(\\는 인식 안되므로 /로 변형해서)'
os.chdir(data_file_path)

with open('hw_2_pickle_data_col.pkl', 'rb') as f:
    data_col = pickle.load(f)
    
GENDER_col_list, DAY_col_list, TIME_col_list, ACT_col_list = data_col

column_list = ['CUS_ID'] + GENDER_col_list + DAY_col_list + TIME_col_list + ACT_col_list


#연월일을 얻어 요일을 구하기 위한 함수 제작.
def get_days(yyyy, mm, dd):
    days = ['MON','TUE','WED','THU', 'FRI', 'SAT', 'SUN']
    return days[datetime.date(yyyy,mm,dd).weekday()]

def data_preprocessing(raw_data_file, label_data_file, save_data_file):
    #os.chdir(data_file_path)
    raw_data = pd.read_csv(raw_data_file, encoding='cp949')
    label_data = pd.read_csv(label_data_file)
    meta_data = pd.merge(raw_data, label_data)
    meta_data['TIME'] = list(map(str,meta_data['TIME_ID']))
    TIME_HOUR = []
    DAY = []
    days = ['MON','TUE','WED','THU', 'FRI', 'SAT', 'SUN']
    df_col = pd.read_pickle('hw_2_pickle_data_col.pkl')

    for timeline in meta_data['TIME']:
        y = int(timeline[:4])
        m = int(timeline[4:6])
        d = int(timeline[6:8])
        h = str(int(timeline[8:]))+'h'
        TIME_HOUR.append(h)
        DAY.append(get_days(y,m,d))

    meta_data['TIME_HOUR'] = TIME_HOUR
    meta_data['DAY'] = DAY
    meta_data.drop(columns={'GENDER', 'AGE', 'AGE_CLASS', 'TIME'})

    days = ['MON','TUE','WED','THU', 'FRI', 'SAT', 'SUN']

    data_for_submission_1 = meta_data.groupby(['CUS_ID','DAY']).size() / meta_data.groupby(['CUS_ID']).size()
    data_for_submission_1 = data_for_submission_1.unstack().replace(np.nan, 0)
    data_for_submission_1 = data_for_submission_1[days]

    data_for_submission_2 = meta_data.groupby(['CUS_ID','TIME_HOUR']).size() / meta_data.groupby(['CUS_ID']).size()
    data_for_submission_2 = data_for_submission_2.unstack().replace(np.nan, 0)

    data_for_submission_3 = meta_data.groupby(['CUS_ID','ACT_NM']).size() / meta_data.groupby(['CUS_ID']).size()
    data_for_submission_3 = data_for_submission_3.unstack().replace(np.nan, 0)

    data_for_submission_4 = meta_data[['CUS_ID', 'GENDER']]
    data_for_submission_4 = data_for_submission_4.sort_values(['CUS_ID']).drop_duplicates(ignore_index = True)

    data_for_submission_final = pd.concat([data_for_submission_1, data_for_submission_2, data_for_submission_3], axis=1, join="outer")
    data_for_submission_final.reset_index()
    data_for_submission_final = pd.merge(data_for_submission_4, data_for_submission_final, how='outer', on='CUS_ID')

    save_df = pd.DataFrame(data_for_submission_final)

# =============================================================================
#     save_df를 작성하시기 바랍니다.
# =============================================================================
    
#    save_df = save_df[column_list]
    save_df.to_csv(save_data_file, header=True, index=False, encoding='cp949')
    
    return save_df
    

if __name__ == "__main__":
    result_df = data_preprocessing('hw_2_raw_data.csv', 'hw_2_label_data.csv', 'hw_2_data_강병우.csv')
    print(result_df.shape)
    print(result_df.head())

(70, 221)
   CUS_ID GENDER       MON       TUE       WED       THU       FRI       SAT  \
0       2      F  0.176644  0.246489  0.160569  0.234109  0.170916  0.011271   
1       3      F  0.220423  0.186444  0.174880  0.214019  0.160292  0.043942   
2       8      M  0.210991  0.190638  0.225237  0.223202  0.149932  0.000000   
3      23      F  0.183091  0.207220  0.198011  0.223061  0.144962  0.017683   
4      32      F  0.220423  0.186444  0.174880  0.214019  0.160292  0.043942   

        SUN        0h  ...        학교  학교커뮤니티   학생/교과교육      학술정보        학원  \
0  0.000000  0.000000  ...  0.000185     0.0  0.000000  0.000924  0.000185   
1  0.000000  0.000000  ...  0.000178     0.0  0.000000  0.001779  0.000890   
2  0.000000  0.000000  ...  0.000678     0.0  0.000000  0.000000  0.000000   
3  0.025972  0.006815  ...  0.000184     0.0  0.147357  0.004237  0.003684   
4  0.000000  0.000000  ...  0.000178     0.0  0.000000  0.001779  0.000890   

   해외지역정보    화장품/미용  화장품/미용 쇼핑몰   화학   환